In [ ]:
! pip install langchain langgraph faiss-cpu sentence-transformers google-generativeai langchain-community

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 📦 Imports
import os
import pickle
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.llms.base import LLM
from pydantic import Field
from typing import Optional

In [2]:
# ✅ Environment config
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

# ✅ Load and preprocess medical literature
loader = TextLoader("medical_literature.txt")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = FAISS.from_documents(docs, embedding_model)
vector_db.save_local("faiss_index")

with open("faiss_index/metadata.pkl", "wb") as f:
    pickle.dump(docs, f)
print("✅ FAISS vector DB created and saved.")


/tmp/ipykernel_11107/194227483.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


✅ FAISS vector DB created and saved.


In [3]:
# ✅ Define Gemini wrapper for LangChain
class GeminiLLM(LLM):
    model_name: str = Field(default="gemini-1.5-flash")

    @property
    def model(self):
        return genai.GenerativeModel(self.model_name)

    def _call(self, prompt, stop=None):
        response = self.model.generate_content(prompt)
        return response.text

    @property
    def _identifying_params(self):
        return {"model_name": self.model_name}

    @property
    def _llm_type(self):
        return "gemini"

llm = GeminiLLM()

In [4]:
# ✅ Tool 1: Retrieve literature from FAISS
def medical_knowledge_agent(query: str) -> str:
    retriever = FAISS.load_local("faiss_index", HuggingFaceEmbeddings(), allow_dangerous_deserialization=True)
    retrieved_docs = retriever.as_retriever().get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    return context if context else "No relevant context found."

# ✅ Tool 2: Analyze symptoms
def symptom_analysis_agent(symptoms: str) -> str:
    prompt = f"Analyze the following symptoms and suggest possible conditions:\n\n{symptoms}"
    response = genai.GenerativeModel("gemini-1.5-flash").generate_content(prompt)
    return response.text

# ✅ Tool 3: Suggest treatments
def treatment_advisor_agent(diagnosis: str, context: str) -> str:
    prompt = f"Based on the diagnosis: {diagnosis}\n\nAnd the medical literature:\n\n{context}\n\nWhat is the best treatment plan?"
    response = genai.GenerativeModel("gemini-1.5-flash").generate_content(prompt)
    return response.text


In [ ]:
# ✅ Agent prompt rules
system_prompt = """
You are an AI Clinical Assistant designed to help doctors. Use the tools to answer medical queries accurately.

- Use tools like `medical_knowledge_agent` when additional context is needed.
- Do not mix final answers with tool calls.
- When context is available, use `treatment_advisor_agent` to propose treatment.
"""

# ✅ Define tools
tools = [
    Tool(name="medical_knowledge_agent", func=medical_knowledge_agent, description="Retrieve medical literature."),
    Tool(name="symptom_analysis_agent", func=symptom_analysis_agent, description="Analyze patient symptoms."),
    Tool(name="treatment_advisor_agent", func=treatment_advisor_agent, description="Suggest treatment options.")
]

# ✅ Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)



/tmp/ipykernel_11107/3312518404.py:18: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [8]:
# ✅ Example usage
query = "I have tiredness with cough give me some advice?"
print(f"\n🔹 **Q:** {query}")
response = agent.invoke(query)
print("🔹 **A:**", response)



🔹 **Q:** I have tiredness with cough give me some advice?


> Entering new AgentExecutor chain...
Question: I have tiredness with cough give me some advice?
Thought: I need to analyze the symptoms first to determine potential causes and then suggest appropriate treatment options.  I'll use the symptom analysis agent.
Action:
```json
{
  "action": "symptom_analysis_agent",
  "action_input": "tiredness and cough"
}
```
Observation: Possible causes of tiredness and cough include the common cold, influenza, bronchitis, pneumonia, allergies, or even underlying conditions like asthma or COPD.  Further information is needed to differentiate.

Thought: The symptom analysis suggests several possibilities.  To recommend treatment, I need more information, but I can offer some general advice based on common causes. I will use the treatment advisor to provide some initial options.
Action:
```json
{
  "action": "treatment_advisor_agent",
  "action_input": "General advice for tiredness and cough, c